## Import statements

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
!pip uninstall pytorch

In [ ]:
!pip install torch

In [10]:
import torch
import torch.nn as nn

## Importing Data
 Below I searched for a suitable dataset to train and test a model on